In [17]:
from __future__ import unicode_literals
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date as dt, timedelta, datetime
from typing import List
from google.cloud import firestore
from functools import reduce
import time
import base64
import sys
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%env GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json"

env: GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json"


In [3]:
DB = firestore.Client.from_service_account_json("/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json")

In [29]:
collection = DB.collection('daily_emotions').stream()

In [30]:
results = []
for document in collection:
    results.append(document.to_dict())

In [31]:
df = pd.DataFrame(data=results)
df.set_index('date', inplace=True)

In [32]:
df

,SUR,ANG,JOY,FEA,TRU,ANT,SAD,DIS
date,,,,,,,,
2020-05-01,0.000000,0.0,0.033493,0.105263,0.014354,0.000000,0.043062,0.081340
2020-05-02,0.000000,0.0,0.015707,0.089005,0.020942,0.000000,0.057592,0.094241
2020-05-03,0.000000,0.0,0.011696,0.076023,0.000000,0.000000,0.040936,0.058480
2020-05-04,0.001368,0.0,0.015048,0.060192,0.009576,0.000000,0.025992,0.067031
2020-05-05,0.000000,0.0,0.010558,0.055807,0.007541,0.001508,0.036199,0.075415
...,...,...,...,...,...,...,...,...
2020-11-26,0.000000,0.0,0.027219,0.046154,0.007101,0.000000,0.018935,0.089941
2020-11-27,0.001099,0.0,0.027473,0.038462,0.008791,0.000000,0.023077,0.151648
2020-11-28,0.000000,0.0,0.030888,0.100386,0.000000,0.000000,0.057915,0.069498


In [35]:
file_name = "../data/main/B3_EMO.pickle"
df.to_pickle(file_name)

In [33]:
len(df[df.isna().any(axis=1)])

24

In [39]:
def removeDupes(tweets):
    return list(set(tweets))

In [45]:
INITIAL_DATE = datetime.strptime('May 1 2020  12:00AM', '%b %d %Y %I:%M%p')
FINAL_DATE = datetime.strptime('Nov 30 2020  12:00AM', '%b %d %Y %I:%M%p')

In [46]:
INITIAL_DATE <= FINAL_DATE

True

In [ ]:
data = []
initdate = INITIAL_DATE
while INITIAL_DATE <= FINAL_DATE:    
    enddate = initdate + timedelta(days=1)
    texts = []
    tweets_col = DB.collection('tweets').where("date", ">=", initdate).where("date", "<=", enddate).order_by("date", "ASCENDING").stream()
    for document in tweets_col:
        texts.append(document.to_dict()["full_text"])
    data.append({"date": initdate.strftime('%Y-%m-%d'), "count": len(removeDupes(texts))})
    initdate = enddate

In [54]:
data = data[:217]

In [58]:
df = pd.DataFrame(data)

df.set_index('date', inplace=True)
df

,count
date,
2020-05-01,209
2020-05-02,191
2020-05-03,171
2020-05-04,731
2020-05-05,663
...,...
2020-11-29,177
2020-11-30,850
2020-12-01,994


In [21]:
tweets = []
for tweet in tweets_col:
    tweets.append(tweet.to_dict())

In [22]:
tweets

[]

In [36]:
emo_df = pd.read_pickle(file_name)

In [37]:
emo_df

,SUR,ANG,JOY,FEA,TRU,ANT,SAD,DIS
date,,,,,,,,
2020-05-01,0.000000,0.0,0.033493,0.105263,0.014354,0.000000,0.043062,0.081340
2020-05-02,0.000000,0.0,0.015707,0.089005,0.020942,0.000000,0.057592,0.094241
2020-05-03,0.000000,0.0,0.011696,0.076023,0.000000,0.000000,0.040936,0.058480
2020-05-04,0.001368,0.0,0.015048,0.060192,0.009576,0.000000,0.025992,0.067031
2020-05-05,0.000000,0.0,0.010558,0.055807,0.007541,0.001508,0.036199,0.075415
...,...,...,...,...,...,...,...,...
2020-11-26,0.000000,0.0,0.027219,0.046154,0.007101,0.000000,0.018935,0.089941
2020-11-27,0.001099,0.0,0.027473,0.038462,0.008791,0.000000,0.023077,0.151648
2020-11-28,0.000000,0.0,0.030888,0.100386,0.000000,0.000000,0.057915,0.069498
